In [1]:
from collections import defaultdict
from collections import deque

In [2]:
def push_button(n,s,m):
    nk = n.copy()
    st = s.copy()
    my = m.copy()
    
    queue = deque()
    low_pulses = 1  #  Count the first pulse from the "button", which hits the "broadcaster"
    high_pulses = 0
    for node in network['broadcaster']['dest']:
        low_pulses += 1
        queue.append(['broadcaster', node, False])  #  False means low pulse, True means high pulse
        
    while queue:
        message = queue.popleft()
        #print(message)
        source = message[0]
        node = message[1]
        pulse_type = message[2]
        if nk[node]['kind'] == 'ff':  #  We've hit a "flipflop" node
            if not pulse_type:  #  Low pulse received
                if st[node]:  #  Flipflop node was on
                    st[node] = False  #  Turn the flipflop node off
                    for dest in network[node]['dest']:
                        low_pulses += 1
                        if dest in network.keys():    
                            queue.append([node, dest, False])  #  Send a high pulse to all its destinations
                        else:
                            pass
                else:  #  Flipflop node was off
                    st[node] = True
                    for dest in network[node]['dest']:
                        high_pulses += 1
                        if dest in network.keys():    
                            queue.append([node, dest, True])
                        else:
                            #print(dest, True)
                            pass
        elif nk[node]['kind'] == 'inv':  #  We have hit a "conjunction" node
            my[node][source] = pulse_type
            if all(my[node].values()):
                output = False
            else:
                output = True
            for dest in network[node]['dest']:
                if output:
                    high_pulses += 1
                else:
                    low_pulses += 1
                if dest in network.keys():
                    queue.append([node, dest, output])  #  Send the output pulse to all its destinations
    
    return st, my, low_pulses, high_pulses

## Part I

In [3]:
with open('input/day-20.input', 'r') as f:
    test = f.read().strip()

In [4]:
partial = dict([[s.split(' -> ')[0], s.split(' -> ')[1].split(', ')] for s in test.split('\n')])

network = {}
for k, v in partial.items():
    if k == 'broadcaster':
        network[k] = {'kind': 'broadcaster', 'dest': v}
    elif k[0] == '%':
        network[k[1:]] = {'kind': 'ff', 'dest': v}
    elif k[0] == '&':
        network[k[1:]] = {'kind': 'inv', 'dest': v}
        
state = dict([[k, False] for k in network.keys() if network[k]['kind'] == 'ff'])  #  Initially all "flipflop" nodes are off
memory = dict([[k, {}] for k in network.keys() if network[k]['kind'] == 'inv'])

for k, v in network.items():
    for node in v['dest']:
        if (node in network) and (network[node]['kind'] == 'inv'):
            memory[node][k] = False

In [5]:
total_low = 0
total_high = 0

for i in range(1000):
    state, memory, low, high = push_button(network, state, memory)
    total_low += low
    total_high += high

total_low*total_high

896998430

## Part II

In [6]:
with open('input/day-20.input', 'r') as f:
    test = f.read().strip()

In [7]:
partial = dict([[s.split(' -> ')[0], s.split(' -> ')[1].split(', ')] for s in test.split('\n')])

network = {}
for k, v in partial.items():
    if k == 'broadcaster':
        network[k] = {'kind': 'broadcaster', 'dest': v}
    elif k[0] == '%':
        network[k[1:]] = {'kind': 'ff', 'dest': v}
    elif k[0] == '&':
        network[k[1:]] = {'kind': 'inv', 'dest': v}
        
state = dict([[k, False] for k in network.keys() if network[k]['kind'] == 'ff'])  #  Initially all "flipflop" nodes are off
memory = dict([[k, {}] for k in network.keys() if network[k]['kind'] == 'inv'])

for k, v in network.items():
    for node in v['dest']:
        if (node in network) and (network[node]['kind'] == 'inv'):
            memory[node][k] = False

In [8]:
def push_button(n,s,m):
    nk = n.copy()
    st = s.copy()
    my = m.copy()
    
    queue = deque()
    low_pulses = 1  #  Count the first pulse from the "button", which hits the "broadcaster"
    high_pulses = 0
    
    rx_hit = False
    
    for node in network['broadcaster']['dest']:
        low_pulses += 1
        queue.append(['broadcaster', node, False])  #  False means low pulse, True means high pulse
        
    while queue:
        message = queue.popleft()
        #print(message)
        source = message[0]
        node = message[1]
        pulse_type = message[2]
        if (node == 'rx') and not pulse_type:
            rx_hit = True
        if (node in nk) and (nk[node]['kind'] == 'ff'):  #  We've hit a "flipflop" node
            if not pulse_type:  #  Low pulse received
                if st[node]:  #  Flipflop node was on
                    st[node] = False  #  Turn the flipflop node off
                    for dest in network[node]['dest']:
                        low_pulses += 1
                        if dest in network.keys():    
                            queue.append([node, dest, False])  #  Send a high pulse to all its destinations
                        else:
                            pass
                else:  #  Flipflop node was off
                    st[node] = True
                    for dest in network[node]['dest']:
                        high_pulses += 1
                        if dest in network.keys():    
                            queue.append([node, dest, True])
                        else:
                            #print(dest, True)
                            pass
        elif (node in nk) and (nk[node]['kind'] == 'inv'):  #  We have hit a "conjunction" node
            my[node][source] = pulse_type
            if all(my[node].values()):
                output = False
            else:
                output = True
            for dest in network[node]['dest']:
                if output:
                    high_pulses += 1
                else:
                    low_pulses += 1
                if dest in network.keys():
                    queue.append([node, dest, output])  #  Send the output pulse to all its destinations
                elif dest == 'rx' and not output:
                    rx_hit = True
    
    return st, my, low_pulses, high_pulses, rx_hit

In [9]:
rx_hit = False

In [10]:
presses = 0

while not rx_hit:
    presses += 1
    state, memory, low, high, rx_hit = push_button(network, state, memory)

KeyboardInterrupt: 